In [ ]:
! git clone https://github.com/Turbo87/utm
%pip install /content/utm/
%pip install geopandas
%pip install geojson
import pandas as pd
import requests
import json
import numpy as np
import utm
import matplotlib.pyplot as plt
from shapely.geometry import MultiPolygon, Polygon
from shapely.ops import cascaded_union
from shapely import wkt
from scipy.interpolate import interp1d
import geojson

In [7]:
def get_influence_polygon(inputJson):
    json_coords = json.loads(inputJson)
    #print(json.dumps(json_coords, indent=4, sort_keys=True))

    lat = json_coords["features"][0]["geometry"]["coordinates"][1]
    #print(lat)
    lon = json_coords["features"][0]["geometry"]["coordinates"][0]
    #print(lon)

    resp = requests.get('http://api.openweathermap.org/data/2.5/weather?lat=' + str(lat) + '&lon=' + str(lon) + '&appid={}')
    if resp.status_code != 200:
        # This means something went wrong.
        raise ApiError('GET /tasks/ {}'.format(resp.status_code))

    #print(resp.text)

    ##Prettify JSON and get Wind Information
    parsed = json.loads(resp.text)
    wind = parsed["wind"]
    wind_speed = parsed["wind"]["speed"]
    wind_dir = parsed["wind"]["deg"]
    #print(wind_speed)
    #print(wind_dir)

    ##Create wind cones
    #Recalculation of movement in km
    #Wind speed is calculated from m/s to km/h via *3.6
    #Fires move 1/10th the km/h of the wind speed (see paper above), thus /10
    wind_speed_move = (float(wind_speed)*3.6)/10 
    #print(wind_speed_move)

    #Recalculation from geographic to arithmetic degrees
    #+180 because the fire goes away from the wind, not into the same direction
    dir_new = (-(wind_dir+180)+450)%360
    #print(dir_new)

    #Calculate x and y
    dx=wind_speed_move*np.cos(np.radians(dir_new))
    #print(dx)
    dy=wind_speed_move*np.sin(np.radians(dir_new))
    #print(dy)

    #Conversion to degrees latitude and longitude
    #The approximate conversions are:
    #Latitude: 1 deg = 110.574 km -> deg = x/110.574
    #Longitude: 1 deg = 111.320*cos(latitude) km -> deg = x/111.320

    #conversion to UTM ('meters')
    utm_coords = utm.from_latlon(lat, lon)

    #add x and y in meters
    newlat=utm_coords[0]+(dx*1000)
    newlon=utm_coords[1]+(dy*1000)

    #Return to WGS84
    pos2 = utm.to_latlon(newlat,newlon,utm_coords[2],utm_coords[3])
    #print(pos2)

    # x, y: coords of forecast position
    y = [lat, pos2[0],47.81765021489975,47.831134971598395]
    x = [lon, pos2[1],13.046951293945312,13.071155548095703]
    # r: radius of uncertainty
    r = [0, 0.001, 0.002, 0.003]
    hours = [0, 1, 2, 3]
    # interpolate
    points_num = 100
    interp_hours = np.linspace(min(hours), max(hours), points_num)
    x = interp1d(hours, x, kind='cubic')(interp_hours)
    y = interp1d(hours, y, kind='cubic')(interp_hours)
    r = interp1d(hours, r, kind='linear')(interp_hours)
    # make polygon
    thetas = np.linspace(0, 2 * np.pi, 360)
    polygon_x = x[:,None] + r[:,None] * np.sin(thetas)
    polygon_y = y[:,None] + r[:,None] * np.cos(thetas)

    # circles
    ps = [Polygon(i) for i in np.dstack((polygon_x, polygon_y))]

    # list of convex hulls of subsequent circles
    n = range(len(ps)-1)
    convex_hulls = [MultiPolygon([ps[i], ps[i+1]]).convex_hull for i in n]

    # Final polygon
    polygons = cascaded_union(convex_hulls)

    #Convert to WKT to convert then to geojson
    g1=wkt.loads(str(polygons))
    g2=geojson.Feature(properties={"Name": "Betroffenheitspolygon"},geometry=g1)

    #print(g2)
    return g2

In [ ]:
input_coords = '{"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Point","coordinates":[13.033218383789062,47.79355334155344]}}]}'
print(input_coords)

In [9]:
polygon=get_influence_polygon(input_coords)

In [ ]:
polygon